# Create High-k Metal Gate Stack Tutorial

This notebook demonstrates how to create a high-k metal gate stack heterostructure with four materials: Si (substrate), SiO2 (gate oxide), HfO2 (high-k dielectric), and TiN (metal gate).

We'll create a representation of the material from the [QuantumATK tutorial](https://docs.quantumatk.com/tutorials/hkmg_builder/hkmg_builder.html) without the amorphous step.

<img src="https://docs.quantumatk.com/_images/update_square_default_stack.png" alt="High-k Metal Gate Stack" width="700"/>


## 1. Configuration Parameters


In [ ]:
# Global parameters
MAX_AREA = 200  # Maximum area for strain matching
MAX_AREA_RATIO_TOL = 0.25  # Maximum area ratio tolerance for strain matching
MAX_ANGLE_TOLERANCE = 0.15  # Maximum angle tolerance for strain matching
MAX_LENGTH_TOLERANCE = 0.15  # Maximum length tolerance for strain matching

# Structure parameters for each layer
STRUCTURE_PARAMS = [
    {
        # Silicon substrate
        "slab_params": {
            "miller_indices": (1, 0, 0),
            "thickness": 4,  # atomic layers
            "vacuum": 5.0,  # Angstroms
            "use_orthogonal_z": True
        },
        "interface_distance": None  # No interface for substrate
    },
    {
        # SiO2 layer
        "slab_params": {
            "miller_indices": (1,0,0),
            "thickness": 3,
            "vacuum": 5.0,
            "use_orthogonal_z": True
        },
        "interface_distance": 2.5  # Distance to Si substrate
    },
    {
        # HfO2 layer
        "slab_params": {
            "miller_indices": (0,0,1),
            "thickness": 4,
            "vacuum": 0.5,
            "xy_supercell_matrix": [[1, 0], [0, 2]],
            "use_orthogonal_z": True
        },
        "interface_distance": 2.5  # Distance to SiO2
    },
    {
        # TiN layer
        "slab_params": {
            "miller_indices": (0,0, 1),
            "thickness": 3,
            "vacuum": 10.0,  # Add vacuum to final layer
            "use_orthogonal_z": True
        },
        "interface_distance": 2.5  # Distance to HfO2
    }
]

INTERFACE_1_INDEX = 11 

### 1.2. Install Packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install` (see [README](../../README.ipynb)).

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip

    await micropip.install('mat3ra-api-examples', deps=False)
    from utils.jupyterlite import install_packages

    await install_packages("specific_examples")

### 1.3. Get input material
In this notebook we will use materials from Standata.

In [ ]:
from mat3ra.made.material import Material
from mat3ra.standata.materials import Materials

materials_1 = [
    Material(Materials.get_by_name_first_match("SiO2")),  # SiO2
    Material(Materials.get_by_name_first_match("Silicon")),  # Si substrate
    ]
materials_2 = [
    Material(Materials.get_by_name_first_match("HfO2")),  # HfO2
    Material(Materials.get_by_name_first_match("TiN")),  # TiN
]

## 2. Create Si/SiO2 Interface
### 2.1. Create Si/SiO2 Interface

In [ ]:
from mat3ra.made.tools.build.slab import SlabConfiguration, get_terminations, create_slab, PymatgenSlabGeneratorParameters
from mat3ra.made.tools.build.interface import (
    InterfaceConfiguration,
    ZSLStrainMatchingParameters,
    ZSLStrainMatchingInterfaceBuilder,
    ZSLStrainMatchingInterfaceBuilderParameters
)
from utils.visualize import visualize_materials as visualize

film_params = STRUCTURE_PARAMS[1]
substrate_params = STRUCTURE_PARAMS[0]

film_config = SlabConfiguration(
    bulk=materials_1[1],
    **film_params["slab_params"]
)

substrate_config = SlabConfiguration(
    bulk=materials_1[0],
    **substrate_params["slab_params"]
)
params = PymatgenSlabGeneratorParameters(symmetrize = False)

film_terminations = get_terminations(film_config, build_parameters=params)
substrate_terminations = get_terminations(substrate_config, build_parameters=params)

film_termination = film_terminations[0]
substrate_termination = substrate_terminations[0]


interface_config = InterfaceConfiguration(
    film_configuration=film_config,
    substrate_configuration=substrate_config,
    film_termination=film_termination,
    substrate_termination=substrate_termination,
    distance=film_params["interface_distance"],
    vacuum=film_params["slab_params"]["vacuum"]
)



builder = ZSLStrainMatchingInterfaceBuilder(
    build_parameters=ZSLStrainMatchingInterfaceBuilderParameters(
        strain_matching_parameters=ZSLStrainMatchingParameters(max_area=MAX_AREA, max_area_ratio_tol=MAX_AREA_RATIO_TOL, max_angle_tol=MAX_ANGLE_TOLERANCE, max_length_tol=MAX_LENGTH_TOLERANCE)
    )
)

interfaces = builder.get_materials(configuration=interface_config)
interface_1 = interfaces[INTERFACE_1_INDEX]

### 2.2. Visualize the Si/SiO2 Interface

In [ ]:
visualize(
    interface_1,
    repetitions=[1, 1, 1],
    title="Si/SiO2 Interface"
)

visualize(
    interface_1,
    repetitions=[1, 1, 1],
    title="Si/SiO2 Interface",
    rotation='-90x'
)


## 3. Add HfO2 Layer
### 3.1. Add a layer with SimpleInterfaceBuilder

In [ ]:
from mat3ra.made.tools.build.interface import SimpleInterfaceBuilder, SimpleInterfaceBuilderParameters

film_params = STRUCTURE_PARAMS[2]

film_config = SlabConfiguration(
    bulk=materials_2[0],
    **film_params["slab_params"]
)

hfo2_slab= create_slab(film_config, build_parameters=params)

film_config = SlabConfiguration(
    bulk=hfo2_slab,
    **film_params["slab_params"]
)

substrate_config = SlabConfiguration(
    bulk=interface_1,
    miller_indices=(0,0,1)
)

film_terminations = get_terminations(film_config, build_parameters=params)
substrate_terminations = get_terminations(substrate_config, build_parameters=params)

print("Film terminations:", film_terminations)
print("Substrate terminations:", substrate_terminations)


interface_config = InterfaceConfiguration(
    film_configuration=film_config,
    substrate_configuration=substrate_config,
    film_termination=film_terminations[0],
    substrate_termination=substrate_terminations[0],
    distance=film_params["interface_distance"],
    vacuum=film_params["slab_params"]["vacuum"]
)

builder = SimpleInterfaceBuilder(build_parameters=SimpleInterfaceBuilderParameters(scale_film=True, build_slabs=False))
interface_2 = builder.get_material(configuration=interface_config)



### 3.2. Visualize the Si/SiO2/HfO2 Interface

In [ ]:
visualize(
    interface_2,
    repetitions=[1, 1, 1],
    title="Si/SiO2/HfO2 Interface"
)

visualize(
    interface_2,
    repetitions=[1, 1, 1],
    title="Si/SiO2/HfO2 Interface",
    rotation='-90x'
)

## 4. Add TiN Layer
### 4.1. Add a layer with SimpleInterfaceBuilder

In [ ]:
from mat3ra.made.tools.build.supercell import create_supercell

film_params = STRUCTURE_PARAMS[3]

interface_2 = create_supercell(interface_2, scaling_factor=[1, 1, -1])

film_config = SlabConfiguration(
    bulk=materials_2[1],
    **film_params["slab_params"]
)

tin_slab = create_slab(film_config, build_parameters=params)

film_config = SlabConfiguration(
    bulk=tin_slab,
    **film_params["slab_params"]
)

substrate_config = SlabConfiguration(
    bulk=interface_2,
    miller_indices=(0,0,1)
)

film_terminations = get_terminations(film_config, build_parameters=params)
substrate_terminations = get_terminations(substrate_config, build_parameters=params)

print("Film terminations:", film_terminations)
print("Substrate terminations:", substrate_terminations)

interface_config = InterfaceConfiguration(
    film_configuration=film_config,
    substrate_configuration=substrate_config,
    film_termination=film_terminations[0],
    substrate_termination=substrate_terminations[0],
    distance=film_params["interface_distance"],
    vacuum=film_params["slab_params"]["vacuum"]
)

builder = SimpleInterfaceBuilder(build_parameters=SimpleInterfaceBuilderParameters(scale_film=True, build_slabs=False))

interface_3 = builder.get_material(configuration=interface_config)
# invert the interface along the z-axis to match example
interface_3 = create_supercell(interface_3, scaling_factor=[1, 1, -1])



### 4.2. Visualize the Si/SiO2/HfO2/TiN Interface

In [ ]:
visualize(
    interface_3,
    repetitions=[1, 1, 1],
    title="Si/SiO2/HfO2/TiN Interface"
)

visualize(
    interface_3,
    repetitions=[1, 1, 1],
    title="Si/SiO2/HfO2/TiN Interface",
    rotation='-90x'
)

## 5. Save final material


In [ ]:
from utils.jupyterlite import download_content_to_file

download_content_to_file(interface_3, "heterostructure_high_k_metal_gate.json")